# Data analysis
2019.04.06 - ?

I get to data analysis in this notebook. This is _part 1_, as I am planning on attacking analysis from another aspect in the future. This notebook looks at the usage of articles in front of specific count and mass nouns, obtained from the prepatory analysis and exploration. However, due to limitations on finding nouns used in each level of interest for four language groups in two different corpora (Arabic L1 from BALC, Arabic L1 from PELIC, Spanish L1 from PELIC, Korean L1 from PELIC), only 1 sample from each level in each language was able to be obtained. Due to this, and due to the fact that I may not have very many interesting or informative findings, I am planning on expanding my analysis in a second part.

## Summary of code
- 

### Set-up

In [1]:
import pandas as pd
import numpy as np
import re 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pprint            # to turn off pretty printing

Pretty printing has been turned OFF


In [2]:
# I tried to use .read_csv, but it didn't work, so I'll continue to use .from_csv
balc = pd.DataFrame.from_csv('../data/balc_targets.csv', index_col=[0])
pelic = pd.DataFrame.from_csv('../private/pelic_targets.csv', index_col=[0])
targets = pd.DataFrame.from_csv('../exploratory-analysis/chosen_targets_new.csv', index_col=[0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


In [3]:
balc.head()

,index,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,1347,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,1171,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,1441,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,1530,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book
4,881,200607798,3,\t\t\t\tCEPA 3 200607798\n\nWhere I going to t...,"Where I going to the India with my family , my...","Where I going to the India with my family , my...","['Where', 'I', 'going', 'to', 'the', 'India', ...",151,0.602649,7.405475,"[(Where, 'WRB'), (I, 'PRP'), (going, 'VBG'), (...","['where', '-PRON-', 'go', 'to', 'the', 'india'...",money


In [4]:
# drop the index column, we don't need it
balc = balc.drop('index', axis=1)
balc.head()

,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book
4,200607798,3,\t\t\t\tCEPA 3 200607798\n\nWhere I going to t...,"Where I going to the India with my family , my...","Where I going to the India with my family , my...","['Where', 'I', 'going', 'to', 'the', 'India', ...",151,0.602649,7.405475,"[(Where, 'WRB'), (I, 'PRP'), (going, 'VBG'), (...","['where', '-PRON-', 'go', 'to', 'the', 'india'...",money


In [5]:
pelic.head()

,answer_id,question_id,anon_id,user_file_id,text,class_code,level_id,native_language,version,toks_re,toks_re_len,toks_nltk,targ
0,1897,188,am1,13293,Adjective\n a lot of traditional desserts\n su...,w,3,Arabic,1,"['adjective', 'a', 'lot', 'of', 'traditional',...",200,"['Adjective', 'a', 'lot', 'of', 'traditional',...",book
1,31692,4194,du3,69515,"I like book called The Last Lecture,because it...",w,4,Arabic,1,"['i', 'like', 'book', 'called', 'the', 'last',...",13,"['I', 'like', 'book', 'called', 'The', 'Last',...",book
2,36798,4778,dp7,78306,Studying English is very crucial because it ...,w,5,Arabic,1,"['studying', 'english', 'is', 'very', 'crucial...",132,"['Studying', 'English', 'is', 'very', 'crucial...",book
3,18572,2544,dg7,41011,? LIST\n\nThe entrance is in front of t...,w,3,Korean,1,"['list', 'the', 'entrance', 'is', 'in', 'front...",124,"['?', 'LIST', 'The', 'entrance', 'is', 'in', '...",book
4,1483,140,eu6,11952,1.The !Kung's most important food is the mongo...,w,4,Korean,1,"['the', 'kung', 's', 'most', 'important', 'foo...",164,"['1', '.', 'The', '!', 'Kung', ""'s"", 'most', '...",book


### Testing regex
I'll be using regular expressions to look for the targets, so below is some testing I did on a UDF and some test phrases to see what I can do with this.

In [6]:
myre = re.compile(r'((\w+ ){,2}\w+ book.*?\b)', re.I)
test = ["I put the book on the counter.", "I put a book on the counter.", "I put the books on the counter",
       "I put some books on the counter.", "I put my books on the counter.", "I put book on the counter.",
       "I put books on the counter.", "I put book and books and the other books on the counter."]

def getArticles(regex, text):
    myre = re.compile(regex)
    search = myre.findall(text)
    articles = []
    for item in search:
        if 'the' in item:
            articles.append((item, 'the'))
        elif 'a' in item or 'an' in item:
            articles.append((item, 'a/an'))
        else:
            articles.append((item, '0'))
    return articles

In [7]:
for s in test:
    print(getArticles(myre, s))

[(('I put the book', 'put '), '0')]
[(('I put a book', 'put '), '0')]
[(('I put the books', 'put '), '0')]
[(('I put some books', 'put '), '0')]
[(('I put my books', 'put '), '0')]
[(('I put book', 'I '), '0')]
[(('I put books', 'I '), '0')]
[(('I put book', 'I '), '0'), (('and books', ''), '0'), (('and the other books', 'the '), '0')]


Well, this is an obvious issue. All of the returns are zero articles, which is simply not true! Let's investigate a little further...

In [8]:
s1 = test[0]
a = myre.findall(s1)
a

[('I put the book', 'put ')]

In [9]:
a[0]
'the' in a[0]

a[0][0]
'the' in a[0][0]

('I put the book', 'put ')

False

'I put the book'

True

So the issue was that I was initially targeting the entire tuple, not the string object of interest, and it was returning 'false' for everything I searched for, thus kicking everything down to the 'else' condition. Let's rewrite getArticles!

In [10]:
def getArticles(regex, text):
    """Uses a regex targeting a noun. Finds all instances of the noun and returns whether the return contains a definite article, an indefinite article, a zero article, or other types of determiners.
    Input: regex (a regex compile), text (can be a single string object or a list of string objects).
    Returns item and type of article in a list format."""
    myre = re.compile(regex)
    look = myre.findall(text)
    other = ['my', 'his', 'her', 'their', 'their', 'our', 'ours', 'its', 
            'this', 'that', 'these', 'those', 'some', 'much', 'many', 'most', 'some', 'any', 'enough',
            'all', "both", 'half', 'either', 'neither', 'each', 'every']
    articles = []
    indef = re.compile(r'\ban?\b')
    for item in look:
        if any(search in item[0] for search in other):
            if 'the' in item[0]:
                articles.append((item, 'the'))
            elif indef.search(item[0]):
                matchobj = indef.search(item[0])
                if matchobj:
                    articles.append((item, 'a/an'))
                else:
                    articles.append((item, 'other'))
            else:
                articles.append((item, 'other'))
        elif 'the' in item[0]:
            articles.append((item, 'the'))
        elif indef.search(item[0]):
            matchobj = indef.search(item[0])
            if matchobj:
                articles.append((item, 'a/an'))
        else:
            articles.append((item, '0'))
    return articles

In [11]:
for s in test:
    print(getArticles(myre, s))

[(('I put the book', 'put '), 'the')]
[(('I put a book', 'put '), 'a/an')]
[(('I put the books', 'put '), 'the')]
[(('I put some books', 'put '), 'other')]
[(('I put my books', 'put '), 'other')]
[(('I put book', 'I '), '0')]
[(('I put books', 'I '), '0')]
[(('I put book', 'I '), '0'), (('and books', ''), '0'), (('and the other books', 'the '), 'the')]


In [12]:
getArticles(myre, s1)

[(('I put the book', 'put '), 'the')]

In [13]:
search = myre.findall(pelic.text[0])
search

[('and international dessert books', 'international '), ('many of dessert books', 'of '), ('and international dessert books', 'international ')]

In [14]:
getArticles(myre, pelic.text[0])

[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]

It may not be perfect, so we'll need to double-check along the way, but this should be a good start. Let's start looking at some nouns!

### Count nouns
In this section, I'll take a look at the targets that are count nouns:
- book 
- place
- park
- week
- story

In [15]:
targets[targets.type == 'count']

,lemma,counts_cepa,counts_pelic,type
37,place,770,1883.0,count
73,park,431,247.0,count
101,week,308,469.0,count
154,story,182,254.0,count
194,book,121,685.0,count


#### 'Book'
myre is already set to look for 'book', so let's just jump into it!

In [16]:
book_pelic = pelic[pelic.targ == 'book']
book_balc = balc[balc.targ == 'book']

In [17]:
book_pelic

,answer_id,question_id,anon_id,user_file_id,text,class_code,level_id,native_language,version,toks_re,toks_re_len,toks_nltk,targ
0,1897,188,am1,13293,Adjective\n a lot of traditional desserts\n su...,w,3,Arabic,1,"['adjective', 'a', 'lot', 'of', 'traditional',...",200,"['Adjective', 'a', 'lot', 'of', 'traditional',...",book
1,31692,4194,du3,69515,"I like book called The Last Lecture,because it...",w,4,Arabic,1,"['i', 'like', 'book', 'called', 'the', 'last',...",13,"['I', 'like', 'book', 'called', 'The', 'Last',...",book
2,36798,4778,dp7,78306,Studying English is very crucial because it ...,w,5,Arabic,1,"['studying', 'english', 'is', 'very', 'crucial...",132,"['Studying', 'English', 'is', 'very', 'crucial...",book
3,18572,2544,dg7,41011,? LIST\n\nThe entrance is in front of t...,w,3,Korean,1,"['list', 'the', 'entrance', 'is', 'in', 'front...",124,"['?', 'LIST', 'The', 'entrance', 'is', 'in', '...",book
4,1483,140,eu6,11952,1.The !Kung's most important food is the mongo...,w,4,Korean,1,"['the', 'kung', 's', 'most', 'important', 'foo...",164,"['1', '.', 'The', '!', 'Kung', ""'s"", 'most', '...",book
5,2738,300,ad1,16211,The statue is situated in the middle of a sea ...,w,5,Korean,1,"['the', 'statue', 'is', 'situated', 'in', 'the...",60,"['The', 'statue', 'is', 'situated', 'in', 'the...",book
6,29216,3881,bt8,66247,...,w,3,Spanish,1,"['types', 'of', 'magazines', 'there', 'are', '...",166,"['Types', 'of', 'magazines', 'There', 'are', '...",book
7,18537,2541,en1,40609,You can get other personal reasons in order to...,w,4,Spanish,1,"['you', 'can', 'get', 'other', 'personal', 're...",90,"['You', 'can', 'get', 'other', 'personal', 're...",book
8,33911,4461,bi4,73652,Learning English has been very hard to underst...,w,5,Spanish,1,"['learning', 'english', 'has', 'been', 'very',...",365,"['Learning', 'English', 'has', 'been', 'very',...",book


In [19]:
getArticles(myre, book_pelic.text[0])
book_pelic.text[0]

[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]

"Adjective\n a lot of traditional desserts\n superior success\n My mother was a wonderful cook\n national and international dessert books\n practiced the desirable recipes\n my favorite food channels\n\n   Phrases of prepositions\n from my mother\n many of dessert books\n on T.V for half an hour a day\n\n\n      Topic:  personal success\n            Making a dessert\n\n        I think I have a personal success in making a lot of traditional desserts from several countries.  There were some reasons for helping my superior success. One of the reasons, I gained this talent from my mother.  She was a wonderful cook especially when she made cakes. In fact, my mother taught me how to prepare German desserts.  For example, I learned the crepes, many kinds of the nut cakes, fruit cakes, and the cake's topping, too. Another reason for my success, I read many of national and international dessert books for several times ago.  I practiced the desirable recipes as soon as I read about them.  Final

In [20]:
index = 0
for x in book_pelic.text:
    print("book_pelic:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

book_pelic: 0
[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]
Adjective
 a lot of traditional desserts
 superior success
 My mother was a wonderful cook
 national and international dessert books
 practiced the desirable recipes
 my favorite food channels

   Phrases of prepositions
 from my mother
 many of dessert books
 on T.V for half an hour a day


      Topic:  personal success
            Making a dessert

        I think I have a personal success in making a lot of traditional desserts from several countries.  There were some reasons for helping my superior success. One of the reasons, I gained this talent from my mother.  She was a wonderful cook especially when she made cakes. In fact, my mother taught me how to prepare German desserts.  For example, I learned the crepes, many kinds of the nut cakes, fruit cakes, and the cake's topping, too. Another reason 

#### A quick look: 'book' in the PELIC sources

##### Indexes 0 - 2: Arabic L1 
###### 0: Arabic L1, Level 3
In regards to using 'book', this writer tends to use the plural 'books' (three times) and (correctly) uses the zero (∅) article in front of these forms. If only accounting for the plural form of 'books', it would appear this writer has perfect acquisition of article usage with plural count nouns. However, taking a glance at some of the other nouns of this type in the passage, we can see that's simply not true (e.g., 'I learned __the__ crepes, many kinds of __the__ nut cakes'). This could, possibly, be due to an effect from the L1 -- Arabic does use the definite article 'al' in front of plural nominal forms.

As I mentioned at the beginning of this notebook, I think it will be more informative to look at nouns and article usage in passages overall instead of just at target nouns. But I will continue this small look into the usage of articles for this target.

Forms: (expected, produced) 
- plural form 'books': (∅, ∅)
- plural form 'books': (∅, ∅)
- plural form 'books': (∅, ∅)

###### 1: Arabic L1, Level 4
This writer uses the ∅ article for a first-mention post-modified noun ('I like ∅ book called The Last Lecture'). Normally, post-modified singular count nouns would take 'the', but because it is a first-mention, I would argue that this is an error with the indefinite article 'a/an'. 

Forms: (expected, produced)
- first-mention singular 'book': (a, ∅)

###### 2: Arabic L1, Level 5
Like the first writer, this writer used the plural form of 'book' ('Studying English in my country has materials and books'). We are not concerned with the lexical error in verb choice -- the plural form 'books' (and 'materials') is used, which warrants the use of the zero article, so the writer is correct in their usage. Overall, this writer seems to use the zero article correctly with the plural form of nouns.

Forms: (expected, produced)
- plural form 'books': (∅, ∅)

##### Indexes 3 - 5: Korean L1 
###### 3: Korean L1, Level 3
Many of the uses of 'book' here are actually a splitting of a nominal form ('book chest' instead of 'bookcase'). The use of articles in front of plural forms is generally as expected (zero article), but at times inconsistent in front of singular forms of count nouns (e.g., 'As you enter the door through the entrance, desk is in front of you.') 

I definitely think it will be more interesting to compare articles at large rather than looking at specific target nouns.

Forms: (expected, produced)
- Irrelevant for target

###### 4: Korean L1, Level 4
The tagging is obviously not perfect in my `getArticles` function -- it incorrectly tagged 'The book' as zero instead of definite, I'm assuming since 'the' is capitalized. It didn't catch the first mention of 'book' in the previous sentence, due to misspelling ('But he used bool source.')

Second-mention book correctly uses 'the', but the writer misses the first-mention indefinite in the previous sentence.

Forms: (expected, produced)
- First mention singular count: (a, ∅)
- Subsequent mention singular count: (the, the)

###### 5: Korean L1, Level 5
Overall, this writer seems to have a good grasp on first-mention article usage, which expects the usage of 'a' or 'an'. 

Forms: (expected, produced)
- First mention singular count: (a, a)

##### Indexes 6 - 8: Spanish L1 
###### 6: Spanish L1, Level 3
Like the first Korean writer, this 'book' is actually not a lone, nominal form ('bookshops'). Overall, this speaker seems to have a handle on the zero article in front of plural nominal forms, but article usage in front of singular nouns seems it may be inconsitent.

Forms: (expected, produced)
- Irrelevant for target

###### 7: Spanish L1, Level 4
In this essay, the writer correctly uses the plural form of 'books' with the zero article. In some cases, it seems there may be some issues with the over-usage of the definite article in front of singular nouns where it is not warranted (e.g., 'the proficiency in English', 'the English').

Forms: (expected, produced)
- plural form 'books': (∅, ∅)

###### 8: Spanish L1, Level 5
In the phrase 'lessons in the book', 'the' is used before the target. I would argue this seems to be used in a sense where the book being talked about is _shared knowledge_, since this seems to be discussing whatever book is used in this class.

The writer seems to have a grasp of zero article in front of plural nouns. That being said, there are a few times throughout the essay where a zero article is used in front of a singular count noun that should actually be pluralized (e.g., 'we have played ∅ games in ∅ team', 'different ∅ form of ∅ words'). 

Forms: (expected, produced)
- Shared knowledge: (the, the)

In [18]:
book_balc

,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book


In [21]:
index = 0
for x in book_balc.Revised_Essay:
    print("book_balc:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

book_balc: 0
[(('wooke in the booke', 'in '), 'the')]
Last summer holiday iam going with my family to perfect weekend. we going to the India. We going to wooke in the booke and go to shopping. And we will go to see Tag Mhale. It was so wonderful. cause it have a nice people and a nice shope. After I finsed in India iam going to Hangare with my family. we spend in Loxx. And iam plaing football and going to swam.

book_balc: 1
[(('friend reading some Bookes', 'reading '), 'other')]
I have just had the perfect holidy. In my holiday I went to dubai, Abu dabi, Sharjah Al-Ain and Fujirah I went there places with my family and my friends. I went the cinama and watched nice movies with my friends, I went malls I buys clothes and play some time with sisters and brothers for example went the sea with my family I just had a wonderfor time I swam in the sea and play with my father and sisters football. I'm very happy becouse I have nice holiday. In a some time I went net coffe set in a nel and dra

#### A quick look: 'book' in the BALC sources

###### 0: Arabic L1, Level 3
There are, unsurprisingly, a lot of orthographic mistakes in this essay. Arabic speakers are well-known to make orthographic mistakes in writing, and all of the CEPA essays are hand-written.

The phrase tagged was 'wooke in the booke' and is tagged as definite. If this phrase is supposed to be 'walk in the park', this would be a correct usage of the definite. However, this seems like a bit of a stretch. If it is something like 'looke in (at?) the booke and go to shopping' (there is some contextual sense here), than this would be an error in production.

There are some interesting article usages in this essay, such as inconsistent usae of 'the India' and '∅ India' (compare with the Arabic term for India, 'al-Hind', which uses the definite article 'al').

Forms: (expected, produced)
- wooke in the booke (I'm not entirely sure what this is supposed to actually be): (?, the)

###### 1: Arabic L1, Level 4
This instance of 'book' is plural and uses a quantifier 'some', which is correct usage. I am more interested in article usage rather than quantifiers, but it may be useful to examine them (at least if they are correctly in place of the zero article in front of plural forms). For example, the writer previously used both the indefinite article and some in front of 'time' ('In a some time').

Forms: (expected, produced)
- plural count 'book(e)s': (∅ or quantifier, quantifier 'some')

###### 2: Arabic L1, Level 5
So this phrase ('holding magazine or newspaper or story book') has the production of the zero article in front of the singular count noun form ('book') but has the sense of plurality ('books'). However, I have a decision here to make: if I take the error to be an error in article production based on the nominal form, the error is that the zero article is used instead of the indefinite article 'a'; if I take the error here to be in the absence of plural '-s' on books, than the correct article was used (∅). 

There are some other interesting article usages in this essay as well, such as 'took us to ∅ Japanese hotel' (use of ∅ where 'a' is expected) and 'a deliscious food' (misuse of a instead of ∅).

Forms: (expected, produced)
- TBD: 
    - If the error is considered to be the absence of plural '-s' on 'book', then: (∅, ∅)
    - If the error is considered to be the absence of indefinite in front of singular 'book', then: (a, ∅)

###### 3: Arabic L1, Level 6
This actually has a common idiom in English, 'don't judge a book by its cover'. They used it correctly, both in form (ignoring orthography) and in meaning. 

There are many instances of articles in this essay, and for the most part, they seem to be used correctly. There's a variety of different uses (first mention, post-modification, etc.), so I think it would be interesting to examine these uses at a larger range than I am currently doing.

Forms: (expected, produced)
- 'don't judge a book by its cover': (a, a)

### Mass nouns
In this section, I'll take a look at the targets that are mass (non-count) nouns:
- time
- sea
- food
- water
- money

In [22]:
targets[targets.type == 'mass']

,lemma,counts_cepa,counts_pelic,type
27,time,984,3595.0,mass
103,sea,301,285.0,mass
117,food,267,3829.0,mass
189,water,125,583.0,mass
321,money,58,1585.0,mass


#### 'Food'

In [23]:
myre = re.compile(r'((\w+ ){,2}\w+ food.*?\b)', re.I)

In [24]:
food_pelic = pelic[pelic.targ == 'food']
food_balc = balc[balc.targ == 'food']

In [26]:
food_pelic

,answer_id,question_id,anon_id,user_file_id,text,class_code,level_id,native_language,version,toks_re,toks_re_len,toks_nltk,targ
9,33276,4355,gk4,72487,The kitchen in my house. There is a yellow inh...,w,3,Arabic,1,"['the', 'kitchen', 'in', 'my', 'house', 'there...",106,"['The', 'kitchen', 'in', 'my', 'house', '.', '...",food
10,23829,3090,db6,57309,Have you think about the energy resources?...,w,4,Arabic,1,"['have', 'you', 'think', 'about', 'the', 'ener...",176,"['Have', 'you', 'think', 'about', 'the', 'ener...",food
11,154,4,fe5,7809,Giving your body the necessary substances to i...,w,5,Arabic,1,"['giving', 'your', 'body', 'the', 'necessary',...",379,"['Giving', 'your', 'body', 'the', 'necessary',...",food
12,15339,2242,cj8,35985,About my kitchen \n\n My kitchen has a...,w,3,Korean,1,"['about', 'my', 'kitchen', 'my', 'kitchen', 'h...",144,"['About', 'my', 'kitchen', 'My', 'kitchen', 'h...",food
13,19052,2586,az0,42810,"On April 15, Tom Higgins opened a new restaura...",w,4,Korean,1,"['on', 'april', 'tom', 'higgins', 'opened', 'a...",187,"['On', 'April', '15', ',', 'Tom', 'Higgins', '...",food
14,2514,276,hb4,15360,"When I visited my relative's house in N.J, I m...",w,5,Korean,1,"['when', 'i', 'visited', 'my', 'relative', 's'...",187,"['When', 'I', 'visited', 'my', 'relative', ""'s...",food
15,30327,4007,ci4,67477,My ideal Apartament...,w,3,Spanish,1,"['my', 'ideal', 'apartament', 'my', 'ideal', '...",171,"['My', 'ideal', 'Apartament', 'My', 'ideal', '...",food
16,28487,3819,bi4,64875,The Impact of an Earthquake\n\n\nEarthquakes a...,w,4,Spanish,1,"['the', 'impact', 'of', 'an', 'earthquake', 'e...",338,"['The', 'Impact', 'of', 'an', 'Earthquake', 'E...",food
17,26033,3426,df4,60529,Adavantages and Disavantages of Both Single an...,w,5,Spanish,1,"['adavantages', 'and', 'disavantages', 'of', '...",372,"['Adavantages', 'and', 'Disavantages', 'of', '...",food


In [25]:
index = 0
for x in food_pelic.text:
    print("food_pelic:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

food_pelic: 0
[(('the food', ''), 'the')]
The kitchen in my house. There is a yellow inhibition around the room. When came to the house  I smelled  the food who my mother cooked. Every day I hear  my mother yelling to my sister because she didn't help her .  In the corner it was the stove I never stayed near it because it's made me I felt  hot and I sweat. There is a window I can see the beautiful back yard from it . There are trees and flours. When the sun light come inside the kitchen it make the room wonderful. This the wonderful place in my house I like it   

food_pelic: 1
[(('preparing food', ''), '0')]
    Have you think about the energy resources? What we can do if the energy resources expiry or unavailable? Can we live without energy? As we know now a days most of thing rely on energy, transportation, preparing food, temperature in building, listening to music, watching TV, traveling over the sea. All these things can't work with out energy. How we can live if the oil is gone.

#### A quick look: 'food' in the PELIC sources

##### Indexes 0 - 2: Arabic L1 
###### 0: Arabic L1, Level 3
This contains the phrase 'I smelled the food who (that) my mother cooked'. Although normally food would not take the definite article in this case, because it is post-modified with the phrase 'who (that) my mother cooked', it is required. Therefore, this is an accurate production of the definite article.

We see correct usage of zero articles in front of plural nouns, but there is a first-mention error ('it was the stove' instead of 'it was (there was) a stove').

Forms: (expected, produced) 
- post-modified mass noun: (the, the)

###### 1: Arabic L1, Level 4
The target ('preparing food') is correctly assigned the zero article. 

Of interesting note is the definite article being used with plurals ('the enegery resources'). Later in the essay, 'how can we live if the oil is gone' is produced, in which 'oil' is used as a mass noun and 'the' is incorrectly used instead of the zero article.

Forms: (expected, produced)
- mass noun: (∅, ∅)

###### 2: Arabic L1, Level 5
One of the uses of 'fast food' is adjectival (modifying 'restaurants'). Otherwise, '∅ fast food' is correctly produced. The last two uses are post-modified, and thus take 'the'. 

Of interesting note: the writer consistently uses 'a good health' instead of '∅ good health', where 'health' is also a mass noun. 

Forms: (expected, produced)
- mass noun: (∅, ∅)
- mass noun: (∅, ∅)
- post-modified: (the, the)
- post-modified: (the, the)

##### Indexes 3 - 5: Korean L1 
###### 3: Korean L1, Level 3
This essay has a lot of first-mention and some second-mention, so there are a lot of cases of the indefinite article in this sample. Most of these uses are correct, although there are some errors in which second-mention nouns are still taking the indefinite 'a'. 

The targeted phrase 'I put all food in a refrigerator', makes me feel weird. This is a case of second-mention taking 'a', which is one issue, but I'm not sure if this should be 'all the food' or not.

Forms: (expected, produced)
- 'all food': (?, ∅))

###### 4: Korean L1, Level 4
Forms: (expected, produced)
- 'good quality food': (∅, ∅)
- 'if the food is so terrible': (the, the)
- 'provide the good food': (∅, the)

###### 5: Korean L1, Level 5
Forms: (expected, produced)
- 'and chose food': (∅, ∅)
- 'pasta and other foods': (∅, ∅)

##### Indexes 6 - 8: Spanish L1 
###### 6: Spanish L1, Level 3
Forms: (expected, produced)
- 'eat my favorite food' -- possessive

###### 7: Spanish L1, Level 4
Forms: (expected, produced)
- regex failed because of whitespace, 'ambulances, food or everything that': (∅, ∅)

###### 8: Spanish L1, Level 5
Forms: (expected, produced)
- 'for buying food': (∅, ∅)

In [27]:
index = 0
for x in food_balc.Revised_Essay:
    print("food_balc:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

food_balc: 0
[(('hotel gived us food', 'gived '), '0'), (('but this food', 'but '), 'other')]
Holiday is very important for anyone because the people can rest and have happy time. Also, people travel and do hobbies in holiday time, so this time for anyone important and useful. I traveled with my family last holiday to the India. This was first time for us to travel india. Befor we always traveled to Europe countries where we enjoy very much and had nice time. The wrost holiday for us. It was last because a lot of reasons. Firstly, we stated at normal hotel. The hotel gived us food, but this food was very bad and made us sicks. Also, some times in the hotel. The electirc closed and no water in ther bath. After that we went to the mountains th India. The mountains were very nice and My family became happy to see this natural place but when we went back to the hotel. Thives stold us and took every thing we have it. Then we went back to the home in the UAE and my family was very sad and we

#### A quick look: 'food' in the BALC sources

###### 0: Arabic L1, Level 3
Forms: (expected, produced)
- (('hotel gived us food', 'gived '), '0'):  (∅, ∅)
- ('but this food', 'but '), 'other') -- possessive, okay

###### 1: Arabic L1, Level 4
Forms: (expected, produced)

###### 2: Arabic L1, Level 5
Forms: (expected, produced)

###### 3: Arabic L1, Level 6
Forms: (expected, produced)